Dashboard

In [2]:
from influxdb import DataFrameClient
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS
import pandas as pd
import numpy as np
import json
from psycopg2.extras import Json
import psycopg2

Wikipedia Infoboxen webscrapen für die S&P 500 Unternehmen

In [ ]:
#-----Password hidden
f=open("PW.txt", "r") 
userPw=[f.readline()]
f.close()
#------------------------SQL Connection

conn = psycopg2.connect(host="mds-dsi-db.postgres.database.azure.com",
                        port="5432",
                        database="finance_data",
                        user="ds22m017",
                        password=str(userPw[0]),#SchachingeR14
                        connect_timeout=3)
cur = conn.cursor()

print("con done")

#------------------------SQL Query
query = "SELECT * FROM wiki_sp_500_companies"
SP_500 = pd.read_sql_query(query, conn)

print("query done")

#------------------------SQL Close
cur.close()
conn.close()

In [86]:
tickers = SP_500['symbol']

In [97]:

df_info = pd.DataFrame([SP_500[SP_500['symbol'] == ticker]['info'].values[0]]  ).T
df_info.columns = [tickers[tickers=="IBM"]]


df_info


symbol,"('IBM',)"
ISIN,IE00B4BNMY34
Type,Public
Founded,1989
Revenue,US$61.6 billion
Website,www.accenture.com
Formerly,Andersen Consulting(1989–2000)
Industry,"['Professional services', 'Information technology', 'Consulting']"
Divisions,"['Strategy & Consulting', ' Technology', ' Operations', ' Song', ' Industry X[4]']"
Traded as,"['NYSE:ACN (Class A)', 'S&P 100 component', 'S&P 500 component']"
Key people,Julie Sweet (Chair and CEO)[1]


Ziehe die Stock Info die durch Kafka Consumer/Producer in die Influxdb geschoben wurden und erzeuge .csv files für die ersten 5 S&P 500 Unternehmen

In [ ]:
# paramenter
display_range = "5d"

In [92]:
for ticker in tickers.head():
    symbol=ticker
    bucket = "DSI_test"
    token ="GMbnHWGhM9p9t9mIjbc1I5KlWir8LJxBDkpMF0SiOa56f1nvLepCEN7iI_5-sR80FA8CvLmf_mHcy8Gc5XYwvA=="
    org="dsi"
    client = InfluxDBClient(url="http://localhost:8086", token=token, org=org)

    query_api = client.query_api()


    query = """from(bucket: "DSI_test")
            |> range(start: -"""+display_range+""")
            |> filter(fn: (r) => r["_measurement"] ==\"""" +symbol+"""\")
            |> filter(fn: (r) => r["_field"] == "Open" or r["_field"] == "Close" or r["_field"] == "High" or r["_field"] == "Low")"""

    result = query_api.query(org=org, query=query)

    results = []
    for table in result:
        for record in table.records:
                results.append((record.get_time(), record.get_measurement(), record.get_field(), record.get_value()))
        
    #write results to excel with name ticker
    f = pd.DataFrame(results, columns=['time', 'symbol', 'field', 'value'])
    f.to_csv(symbol+'.csv', index=False)

Hier noch mal in parametriserter Form

In [98]:
#parameter
display_range = "2d"
symbol = "IBM"

In [99]:
bucket = "DSI_test"
token ="GMbnHWGhM9p9t9mIjbc1I5KlWir8LJxBDkpMF0SiOa56f1nvLepCEN7iI_5-sR80FA8CvLmf_mHcy8Gc5XYwvA=="
org="dsi"
client = InfluxDBClient(url="http://localhost:8086", token=token, org=org)

query_api = client.query_api()


query = """from(bucket: "DSI_test")
        |> range(start: -"""+display_range+""")
        |> filter(fn: (r) => r["_measurement"] ==\"""" +symbol+"""\")
        |> filter(fn: (r) => r["_field"] == "Open" or r["_field"] == "Close" or r["_field"] == "High" or r["_field"] == "Low")"""

result = query_api.query(org=org, query=query)

In [100]:
results = []
for table in result:
  for record in table.records:
    results.append((record.get_time(), record.get_measurement(), record.get_field(), record.get_value()))


results

[(datetime.datetime(2023, 1, 20, 18, 18, tzinfo=datetime.timezone.utc),
  'IBM',
  'Close',
  140.75),
 (datetime.datetime(2023, 1, 20, 18, 19, tzinfo=datetime.timezone.utc),
  'IBM',
  'Close',
  140.5700073242),
 (datetime.datetime(2023, 1, 20, 18, 20, tzinfo=datetime.timezone.utc),
  'IBM',
  'Close',
  140.5899963379),
 (datetime.datetime(2023, 1, 20, 18, 21, tzinfo=datetime.timezone.utc),
  'IBM',
  'Close',
  140.6199951172),
 (datetime.datetime(2023, 1, 20, 18, 22, tzinfo=datetime.timezone.utc),
  'IBM',
  'Close',
  140.6150054932),
 (datetime.datetime(2023, 1, 20, 18, 23, tzinfo=datetime.timezone.utc),
  'IBM',
  'Close',
  140.6900024414),
 (datetime.datetime(2023, 1, 20, 18, 24, tzinfo=datetime.timezone.utc),
  'IBM',
  'Close',
  140.7200012207),
 (datetime.datetime(2023, 1, 20, 18, 25, tzinfo=datetime.timezone.utc),
  'IBM',
  'Close',
  140.7250061035),
 (datetime.datetime(2023, 1, 20, 18, 26, tzinfo=datetime.timezone.utc),
  'IBM',
  'Close',
  140.6600036621),
 (datetim

In [101]:
#make results a dataframe
df = pd.DataFrame(results, columns=['Date',"Symbol", 'OHLC', "Value"])
df['Date'] = pd.to_datetime(df['Date'])
#if OHLC is Open, High, Low, or Close, then make it a column
df = df.pivot(index='Date', columns='OHLC', values='Value')
df.head()

OHLC,Close,High,Low,Open
Date,,,,
2023-01-20 18:18:00+00:00,140.750000,140.779999,140.735001,140.759995
2023-01-20 18:19:00+00:00,140.570007,140.745895,140.570007,140.740005
2023-01-20 18:20:00+00:00,140.589996,140.589996,140.550003,140.550003
2023-01-20 18:21:00+00:00,140.619995,140.660004,140.580002,140.589996
2023-01-20 18:22:00+00:00,140.615005,140.619995,140.559998,140.615005


In [103]:
#build plotly stock graph
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])
#add title with symbol
fig.update_layout(title_text=symbol)
#y axis 

fig.show()